<a href="https://colab.research.google.com/github/sreent/machine-learning/blob/main/Regularisation/Regularisation%20Code%20Walk%20Through.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularisation: Code Walk Through

This notebook provides a step-by-step computational walkthrough of **L2 Regularisation (Ridge Regression)** applied to logistic regression.

## What You'll Learn

- How **overfitting** manifests with large weights in high-degree polynomial features
- How to add the **L2 penalty term** ($\lambda ||\vec{w}||^2$) to the loss function
- How to compute the **regularised gradient**: $\nabla J = \Phi^T(p - y) + 2\lambda\vec{w}$
- How regularisation **shrinks weights** to prevent overfitting
- How the **regularisation parameter λ** controls the bias-variance tradeoff
- How to visualise decision boundaries with different λ values

## 1. Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit  # Numerically stable sigmoid
from sklearn.preprocessing import PolynomialFeatures

# Set random seed for reproducibility
np.random.seed(42)

## 2. Generate Synthetic Binary Classification Data

We'll use the same mixture dataset pattern from the lecture slides - two classes with non-linear separation.

In [ ]:
# Generate two-class data (matching lecture slides pattern)
m = 25  # samples per class
n = 2   # features

np.random.seed(0)

# Class 0: centered around (1.5, -1.5)
class_0 = np.hstack((
    1.5 + np.random.randn(m, 1),
    -1.5 + np.random.randn(m, 1)
))

# Class 1: centered around (-1.5, 1.5)
class_1 = np.hstack((
    -1.5 + np.random.randn(m, 1),
    1.5 + np.random.randn(m, 1)
))

# Combine into training set
X = np.vstack((class_0, class_1))  # shape (2m, 2)
y = np.concatenate([np.zeros(m), np.ones(m)])  # shape (2m,)

print(f"Dataset shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Class distribution: {np.bincount(y.astype(int))}")

## 3. Visualize the Data

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X[y == 0, 0], X[y == 0, 1],
            c='orange', label='Class 0', edgecolors='k', s=50)
plt.scatter(X[y == 1, 0], X[y == 1, 1],
            c='skyblue', label='Class 1', edgecolors='k', s=50)
plt.xlabel('$x_1$', fontsize=12)
plt.ylabel('$x_2$', fontsize=12)
plt.title('Binary Classification Data', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. The Overfitting Problem

Let's first demonstrate **why we need regularisation** by training a logistic regression model with high-degree polynomial features **without regularisation**.

### Create Polynomial Features (Degree 8)

In [ ]:
# Create 8th degree polynomial features (like in lecture slides)
degree = 8
poly = PolynomialFeatures(degree=degree, include_bias=False)
X_poly = poly.fit_transform(X)

print(f"Original features: {X.shape[1]}")
print(f"Polynomial features (degree {degree}): {X_poly.shape[1]}")
print(f"\nFeature names: {poly.get_feature_names_out(['x1', 'x2'])}")

### Prepare Design Matrix

In [ ]:
# Add bias column: Φ = [1 | X]
Phi = np.c_[np.ones(len(X_poly)), X_poly]
print(f"Design matrix Φ shape: {Phi.shape}")
print(f"First row (bias + features): {Phi[0][:5]}... (showing first 5 values)")

### Initialize Weights

In [ ]:
np.random.seed(42)
weights = np.zeros(Phi.shape[1])  # Initialize to zeros
print(f"Initial weights shape: {weights.shape}")
print(f"Initial weights: {weights[:5]}... (all zeros)")

## 5. Step-by-Step: One Gradient Descent Iteration WITHOUT Regularisation

Let's walk through **one iteration** to understand the standard logistic regression update (without regularisation).

### Step 1: Compute Scores (Logits)

In [ ]:
scores = Phi @ weights
print(f"Scores shape: {scores.shape}")
print(f"First 5 scores: {scores[:5]}")

### Step 2: Apply Sigmoid to Get Probabilities

$$P(y=1|\vec{x}, \vec{w}) = \sigma(\vec{x}^T \times \vec{w}) = \frac{1}{1 + e^{-\vec{x}^T \times \vec{w}}}$$

In [ ]:
probabilities = expit(scores)
print(f"Probabilities shape: {probabilities.shape}")
print(f"First 5 probabilities: {probabilities[:5]}")
print(f"All probabilities in [0,1]: {np.all((probabilities >= 0) & (probabilities <= 1))}")

### Step 3: Compute Gradient (WITHOUT Regularisation)

**Standard gradient (no regularisation):**

$$\nabla_{\vec{w}} J = \Phi^T (\vec{p} - \vec{y})$$

In [ ]:
errors = probabilities - y
gradient_no_reg = Phi.T @ errors

print(f"Gradient shape: {gradient_no_reg.shape}")
print(f"Gradient (no regularisation): {gradient_no_reg[:5]}")
print(f"Gradient magnitude: {np.linalg.norm(gradient_no_reg):.6f}")

## 6. Now Add L2 Regularisation!

### The Key Addition: Regularisation Gradient Term

**Total Cost with L2:**

$$J(\vec{w}) = -\sum [y \log p + (1-y)\log(1-p)] + \lambda ||\vec{w}||^2$$

**Regularised Gradient:**

$$\nabla_{\vec{w}} J = \Phi^T (\vec{p} - \vec{y}) + 2\lambda\vec{w}$$

**Important:** We don't regularise the bias term (intercept)!

### Step 4: Compute L2 Regularisation Term

The regularisation adds $2\lambda\vec{w}$ to the gradient (excluding the bias term).

In [ ]:
# Set regularisation strength
lambda_reg = 1.0

# Create regularisation vector (exclude bias w[0])
reg_vector = weights.copy()
reg_vector[0] = 0.0  # Don't regularise the intercept

# Compute regularisation gradient term: 2λw
gradient_reg_term = 2 * lambda_reg * reg_vector

print(f"Regularisation term (2λw): {gradient_reg_term[:5]}")
print(f"Note: First element (bias) is {gradient_reg_term[0]} (not regularised)")

### Step 5: Compute Total Gradient (WITH Regularisation)

$$\nabla_{\vec{w}} J = \underbrace{\Phi^T (\vec{p} - \vec{y})}_{\text{base gradient}} + \underbrace{2\lambda\vec{w}}_{\text{L2 penalty}}$$

In [ ]:
# Total gradient = base gradient + regularisation term
gradient_with_reg = gradient_no_reg + gradient_reg_term

print("\n" + "="*60)
print("COMPARISON: Gradient WITHOUT vs WITH Regularisation")
print("="*60)
print(f"\nGradient (no reg):   {gradient_no_reg[:3]}")
print(f"Reg term (2λw):      {gradient_reg_term[:3]}")
print(f"Gradient (with reg): {gradient_with_reg[:3]}")
print(f"\nMagnitude without reg: {np.linalg.norm(gradient_no_reg):.6f}")
print(f"Magnitude with reg:    {np.linalg.norm(gradient_with_reg):.6f}")

### Step 6: Update Weights

$$\vec{w}_{\text{new}} = \vec{w}_{\text{old}} - \alpha \nabla_{\vec{w}} J$$

In [ ]:
learning_rate = 0.1
weights_new = weights - learning_rate * gradient_with_reg

print(f"Old weights: {weights[:3]}")
print(f"New weights: {weights_new[:3]}")
print(f"Weight change: {weights_new[:3] - weights[:3]}")

## 7. Full Gradient Descent Function with L2 Regularisation

In [ ]:
def gradient_descent_l2(Phi, y, lambda_reg=0.0, learning_rate=0.1,
                        max_iter=1000, tol=1e-6):
    """
    Gradient descent with L2 regularisation.

    Parameters:
    -----------
    Phi : design matrix with bias column
    y : target labels
    lambda_reg : L2 regularisation strength (λ)
    learning_rate : step size (α)
    max_iter : maximum iterations
    tol : convergence tolerance

    Returns:
    --------
    weights : learned weights
    loss_history : loss at each iteration
    weight_history : weights at each iteration
    """
    # Initialize weights
    weights = np.zeros(Phi.shape[1])
    loss_history = []
    weight_history = [weights.copy()]

    for iteration in range(max_iter):
        # Forward pass
        scores = Phi @ weights
        probabilities = expit(scores)

        # Compute loss (NLL + L2 penalty)
        epsilon = 1e-15
        p_safe = np.clip(probabilities, epsilon, 1 - epsilon)
        nll = -np.sum(y * np.log(p_safe) + (1 - y) * np.log(1 - p_safe))

        # Add L2 penalty (don't regularise bias)
        l2_penalty = lambda_reg * np.sum(weights[1:]**2)
        total_loss = nll + l2_penalty
        loss_history.append(total_loss)

        # Compute gradient
        errors = probabilities - y
        gradient_base = Phi.T @ errors

        # Add L2 regularisation term (exclude bias)
        reg_vector = weights.copy()
        reg_vector[0] = 0.0  # Don't regularise intercept
        gradient_reg = 2 * lambda_reg * reg_vector

        gradient = gradient_base + gradient_reg

        # Check convergence
        if np.linalg.norm(gradient) < tol:
            break

        # Update weights
        weights = weights - learning_rate * gradient
        weight_history.append(weights.copy())

    return weights, loss_history, weight_history, iteration + 1

## 8. Train Models with Different λ Values

Let's train three models to see the effect of regularisation:
1. **λ = 0** (No regularisation) → Overfitting
2. **λ = 1** (Moderate regularisation) → Balanced
3. **λ = 100** (Heavy regularisation) → Underfitting

In [ ]:
# Train models with different λ values
lambdas = [0, 1.0, 100]
results = {}

print("="*70)
print("Training Models with Different λ Values")
print("="*70)

for lam in lambdas:
    print(f"\nTraining with λ = {lam}...")
    weights, loss_hist, weight_hist, n_iter = gradient_descent_l2(
        Phi, y, lambda_reg=lam, learning_rate=0.1, max_iter=2000
    )

    results[lam] = {
        'weights': weights,
        'loss_history': loss_hist,
        'weight_history': weight_hist,
        'n_iter': n_iter
    }

    # Compute predictions
    probs = expit(Phi @ weights)
    preds = (probs >= 0.5).astype(int)
    accuracy = np.mean(preds == y)

    # Weight magnitude
    weight_magnitude = np.linalg.norm(weights)

    print(f"  Converged in {n_iter} iterations")
    print(f"  Final loss: {loss_hist[-1]:.4f}")
    print(f"  Accuracy: {accuracy:.3f}")
    print(f"  Weight magnitude: {weight_magnitude:.2f}")
    print(f"  Max |weight|: {np.max(np.abs(weights[1:])):.2f}")

print("\n" + "="*70)
print("OBSERVATION: As λ increases, weight magnitudes decrease!")
print("="*70)

## 9. Visualize Loss Convergence

In [ ]:
plt.figure(figsize=(12, 5))

for lam in lambdas:
    loss_hist = results[lam]['loss_history']
    plt.plot(loss_hist, linewidth=2, label=f'λ = {lam}')

plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Total Loss (NLL + λ||w||²)', fontsize=12)
plt.title('Loss Convergence with Different λ Values', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

## 10. Compare Weight Magnitudes

Let's see how regularisation **shrinks weights** to prevent overfitting.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

for idx, lam in enumerate(lambdas):
    ax = axes[idx]
    weights = results[lam]['weights']

    # Plot weights (excluding bias)
    ax.bar(range(1, len(weights)), weights[1:], alpha=0.7)
    ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    ax.set_xlabel('Weight Index', fontsize=11)
    ax.set_ylabel('Weight Value', fontsize=11)
    ax.set_title(f'λ = {lam}\n||w|| = {np.linalg.norm(weights):.2f}',
                 fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("OBSERVATION:")
print("• λ = 0: Large weights → complex boundary → overfitting")
print("• λ = 1: Moderate weights → smooth boundary → good fit")
print("• λ = 100: Tiny weights → simple boundary → underfitting")

In [ ]:
print("\nSample Weights Comparison:")
print(f"Feature x1²: λ=0: {results[0]['weights'][3]:.2f}, "
      f"λ=1: {results[1.0]['weights'][3]:.2f}, "
      f"λ=100: {results[100]['weights'][3]:.2f}")

## 11. Visualize Decision Boundaries

In [ ]:
# Create mesh for decision boundary visualization
x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 200),
                        np.linspace(x2_min, x2_max, 200))

# Prepare mesh
X_mesh = np.c_[xx1.ravel(), xx2.ravel()]
X_mesh_poly = poly.transform(X_mesh)
Phi_mesh = np.c_[np.ones(X_mesh_poly.shape[0]), X_mesh_poly]

# Plot decision boundaries
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, lam in enumerate(lambdas):
    ax = axes[idx]
    weights = results[lam]['weights']

    # Compute probabilities for mesh
    probs_mesh = expit(Phi_mesh @ weights).reshape(xx1.shape)

    # Plot contours
    ax.contourf(xx1, xx2, probs_mesh, levels=20, cmap='RdBu_r', alpha=0.6)
    ax.contour(xx1, xx2, probs_mesh, levels=[0.5], colors='black', linewidths=2.5)

    # Plot data
    ax.scatter(X[y == 0, 0], X[y == 0, 1], c='orange',
               edgecolors='k', s=50, alpha=0.8, label='Class 0')
    ax.scatter(X[y == 1, 0], X[y == 1, 1], c='skyblue',
               edgecolors='k', s=50, alpha=0.8, label='Class 1')

    # Compute accuracy
    probs = expit(Phi @ weights)
    preds = (probs >= 0.5).astype(int)
    accuracy = np.mean(preds == y)

    # Title
    if lam == 0:
        title = f'λ = {lam} (No Regularisation)\nAcc = {accuracy:.2f} - Overfitting'
    elif lam == 1:
        title = f'λ = {lam} (Moderate)\nAcc = {accuracy:.2f} - Balanced ✓'
    else:
        title = f'λ = {lam} (Heavy)\nAcc = {accuracy:.2f} - Underfitting'

    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xlabel('$x_1$', fontsize=11)
    ax.set_ylabel('$x_2$', fontsize=11)
    ax.legend(fontsize=9)
    ax.axis('equal')
    ax.set_xlim(x1_min, x1_max)
    ax.set_ylim(x2_min, x2_max)

plt.tight_layout()
plt.show()

## 12. Comparison with scikit-learn

Let's validate our implementation against scikit-learn.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Test with λ = 1.0
lambda_test = 1.0

# Our implementation
our_weights = results[lambda_test]['weights']
our_probs = expit(Phi @ our_weights)
our_accuracy = np.mean((our_probs >= 0.5).astype(int) == y)

# scikit-learn (C = 1/λ)
sklearn_model = LogisticRegression(penalty='l2', C=1.0/lambda_test,
                                   max_iter=2000, random_state=42)
sklearn_model.fit(X_poly, y)
sklearn_accuracy = sklearn_model.score(X_poly, y)
sklearn_weights = np.concatenate([sklearn_model.intercept_, sklearn_model.coef_[0]])

print("="*70)
print("COMPARISON: Our Implementation vs scikit-learn")
print("="*70)
print(f"\nλ = {lambda_test}")
print(f"\nOur accuracy:      {our_accuracy:.4f}")
print(f"sklearn accuracy:  {sklearn_accuracy:.4f}")
print(f"\nDifference: {abs(our_accuracy - sklearn_accuracy):.4f}")

print(f"\nOur weight magnitude:     {np.linalg.norm(our_weights):.2f}")
print(f"sklearn weight magnitude: {np.linalg.norm(sklearn_weights):.2f}")

print("\n" + "="*70)
if abs(our_accuracy - sklearn_accuracy) < 0.02:
    print("✓ Results match! L2 regularisation implemented correctly.")
else:
    print("⚠ Results differ. Check implementation.")
print("="*70)

## 13. The Bias-Variance Tradeoff

Let's create a visualization showing how λ affects the bias-variance tradeoff.

In [ ]:
# Test a range of λ values
lambda_range = np.logspace(-3, 3, 20)  # From 0.001 to 1000
accuracies = []
weight_mags = []

for lam in lambda_range:
    weights, _, _, _ = gradient_descent_l2(Phi, y, lambda_reg=lam,
                                           learning_rate=0.1, max_iter=2000)
    probs = expit(Phi @ weights)
    acc = np.mean((probs >= 0.5).astype(int) == y)
    accuracies.append(acc)
    weight_mags.append(np.linalg.norm(weights))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Accuracy vs λ
ax = axes[0]
ax.plot(lambda_range, accuracies, 'o-', linewidth=2, markersize=6)
ax.axvline(1.0, color='red', linestyle='--', linewidth=2, label='λ = 1.0 (optimal)')
ax.set_xscale('log')
ax.set_xlabel('Regularisation Strength (λ)', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Accuracy vs λ: Finding the Sweet Spot', fontsize=14)
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11)

# Add annotations
ax.text(0.001, 0.5, 'Overfitting\n(High Variance)', fontsize=10,
        ha='center', bbox=dict(boxstyle='round', facecolor='pink', alpha=0.5))
ax.text(1000, 0.5, 'Underfitting\n(High Bias)', fontsize=10,
        ha='center', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

# Plot 2: Weight magnitude vs λ
ax = axes[1]
ax.plot(lambda_range, weight_mags, 's-', linewidth=2, markersize=6, color='green')
ax.set_xscale('log')
ax.set_xlabel('Regularisation Strength (λ)', fontsize=12)
ax.set_ylabel('Weight Magnitude (||w||)', fontsize=12)
ax.set_title('Weight Shrinkage Effect of Regularisation', fontsize=14)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKEY INSIGHT:")
print("• Small λ → Large weights → Complex boundary → Overfitting")
print("• Optimal λ → Moderate weights → Smooth boundary → Good fit")
print("• Large λ → Small weights → Simple boundary → Underfitting")

## Summary

We've walked through all the computational steps of L2 Regularisation:

1. ✅ **Demonstrated overfitting** - high-degree polynomials create large weights
2. ✅ **Added L2 penalty** - modified loss function: $J = NLL + \lambda||\vec{w}||^2$
3. ✅ **Computed regularised gradient** - added $2\lambda\vec{w}$ term (excluding bias)
4. ✅ **Implemented gradient descent** with regularisation
5. ✅ **Visualised weight shrinkage** - regularisation reduces weight magnitudes
6. ✅ **Explored bias-variance tradeoff** - λ controls model complexity
7. ✅ **Validated implementation** - matched scikit-learn results

### Key L2 Regularisation Concepts

| Concept | Description |
|---------|-------------|
| **L2 Penalty** | $\lambda||\vec{w}||^2 = \lambda\sum_{j=1}^{d} w_j^2$ |
| **Regularised Gradient** | $\nabla J = \Phi^T(p - y) + 2\lambda\vec{w}$ |
| **Weight Shrinkage** | Larger λ → Smaller weights |
| **Bias-Variance** | Small λ → High variance (overfit)<br>Large λ → High bias (underfit) |
| **Exclude Bias** | Don't regularise intercept term $w_0$ |

### When to Use L2 Regularisation

✅ **Use L2 when:**
- Using polynomial or interaction features
- Model has many parameters relative to data
- Observing overfitting (train >> test accuracy)
- Want to keep all features but reduce their influence

❌ **Don't use L2 when:**
- Model is already underfitting
- Dataset is very large relative to features
- Need sparse feature selection (use L1 instead)

### Choosing λ

Use **cross-validation** to find optimal λ:
1. Try range: [0.001, 0.01, 0.1, 1, 10, 100]
2. Train model on training folds
3. Validate on validation fold
4. Select λ with best validation accuracy
5. Retrain on full training set with optimal λ